# Testing homemade layers

This notebook contains tests for all the homemade layers. The tests takes some samples from the mni-
st dataset as a minibatch and compares the results from running this minibatch through the test CNN
to running the same minibatch through the homemade layers.

In [1]:
#importing relevant libraries

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, TensorDataset, Subset
import torch.optim as optim
import torch.nn.functional as F
import os

from CNN_small_architecture import CNNSmall
from CNN_layers import conv_homemade
from CNN_layers import maxpool_homemade
from CNN_layers import batchnorm_homemade
from CNN_layers import linear_layer_homemade
from CNN_layers import elu_homemade

In [2]:
def tokenize(num):
    if num == 1:
        return torch.tensor(np.array([1., 0.]))
    else:
        return torch.tensor(np.array([0., 1.]))

MNIST_test = datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_set = [[data[0], tokenize(data[1])] for data in MNIST_test if data[1] in [1,2]]

batch_size = 2
test_loader = DataLoader(test_set, batch_size=batch_size)

/home/emil/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
# takes a 4-dimensional tensor from the dataloader (batch_size x channel_size x height x width)
# and turns it into a list of lists of (height x width) numpy arrays
def transform_input(input_batch):
    return list(list(input_batch.detach().numpy()))

# compares two lists of batches with same amount of channels 
# and returns the summed absoute loss for each batch over the channels
def compare(x, y):
    b_diff = []
    batch_size = len(x)

    for b in range(batch_size):
        c_diff = []
        channels = len(x[b])

        for c in range(channels):
            diff = np.sum(np.absolute(x[b][c] - y[b][c]))
            c_diff.append(diff)
        
        b_diff.append(c_diff)
    
    return b_diff

In [4]:
model_original = CNNSmall()
model_original.eval()

path = "CNN_small"
load = True

if load and os.path.isfile(path):
    model_original.load_state_dict(torch.load(path))

In [5]:
input_original_test, label_test = next(iter(test_loader))
input_homemade_test = transform_input(input_batch=input_original_test)

In [6]:
def create_conv_homemade(model_conv):
    weights = model_conv.weight
    biases = model_conv.bias.detach().numpy()

    out_c, in_c, r, c = weights.shape
    conv1_filters = []

    for f in range(out_c):
        filter_ = []
        kernels = []

        for kernel in list(weights[f,:,:,:]):
            kernels.append(kernel.detach().numpy())

        filter_.append(kernels)
        filter_.append(biases[f])
        conv1_filters.append(filter_)
    
    return conv_homemade.Conv(filters=conv1_filters, in_channels=in_c)


In [7]:
def create_batchnorm_homemade(model_batchnorm):
    weights = model_batchnorm.weight
    biases = model_batchnorm.bias
    running_mean = model_batchnorm.running_mean
    running_var = model_batchnorm.running_var

    return batchnorm_homemade.BatchNorm(weights=weights, biases=biases, running_mean = running_mean, running_var = running_var)

In [8]:
def create_maxpool_homemade(model_maxpool):
    kernel_size = model_maxpool.kernel_size
    stride = model_maxpool.stride
    if type(model_maxpool.padding) == int:
        padding = (model_maxpool.padding, model_maxpool.padding)
    else:
        padding = model_maxpool.padding
    
    return maxpool_homemade.MaxPool(kernel_size=kernel_size, stride=stride, padding=padding)
    


In [9]:
elu_homemade = elu_homemade.ELU()
conv1_homemade = create_conv_homemade(model_conv= model_original.conv1)
batchnorm1_homemade = create_batchnorm_homemade(model_batchnorm= model_original.batchNorm1)
maxpool1_homemade = create_maxpool_homemade(model_maxpool= model_original.maxPool1)
conv2_homemade = create_conv_homemade(model_conv= model_original.conv2)
batchnorm2_homemade = create_batchnorm_homemade(model_batchnorm= model_original.batchNorm2)
maxpool2_homemade = create_maxpool_homemade(model_original.maxPool2)
linear_homemade = linear_layer_homemade.linear_layer(model_original.lin.weight,model_original.lin.bias,2)

STRIDE ER PRÆDEFINERET
STRIDE ER PRÆDEFINERET


In [33]:
m = nn.ELU()
input = torch.randn(1,1,3,3)
Output = m(input)
Output_homemade = elu_homemade(transform_input(input))
Output_homemade

[[array([[-0.81710519,  0.82246125,  0.47514841],
         [-0.38121629, -0.45230168,  0.04021471],
         [-0.86837724, -0.02361894, -0.72806326]])]]

## Testing the first convolutional layer

In [11]:
# homemade conv1 filter on test
out_homemade = conv1_homemade(input_homemade_test)
# original conv1 filter on test
out_original = model_original.conv1(input_original_test)

print("Convolutional layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

Convolutional layer 1 error over out channels: 


[[3.2643608398608137e-06, 2.7843965184626285e-06, 1.730350354417487e-06],
 [5.192771076883984e-06, 4.784680839725608e-06, 3.57525076902232e-06]]

## Testing the first Batchnorm layer

In [12]:
# homemade batchNorm1 filter on test
out_homemade = batchnorm1_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.batchNorm1(out_original)

print("BatchNorm layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

BatchNorm layer 1 error over out channels: 


[[4.727607301902026e-05, 5.4140109568834305e-05, 5.2090734243392944e-05],
 [5.4426491260528564e-05, 6.492342799901962e-05, 6.741611286997795e-05]]

## Testing the first Maxpool layer

In [13]:
# homemade MaxPool filter on test
out_homemade = maxpool1_homemade(input_batch=out_homemade)
# original MaxPool filter on test
out_original = model_original.maxPool1(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


/home/emil/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[[1.290654472541064e-05, 1.4092773199081421e-05, 1.3031065464019775e-05],
 [1.6646459698677063e-05, 1.4612916857004166e-05, 1.897662878036499e-05]]

## Testing the second Convolutional layer

In [14]:
# TODO: test the second convolutional layer

# homemade batchNorm1 filter on test
out_homemade = conv2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.conv2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


[[8.414660649894157e-06,
  1.1579165318589824e-05,
  1.1777937392112858e-05,
  1.0706716826278107e-05,
  7.7276939932161e-06],
 [1.1472367756532043e-05,
  1.597145474213635e-05,
  1.6001494683975936e-05,
  1.108905038547936e-05,
  1.139260929336472e-05]]

## Testing the second Batchnorm layer

In [15]:
# homemade batchNorm1 filter on test
out_homemade = batchnorm2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.batchNorm2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


[[1.58846378326416e-05,
  1.6979523934423923e-05,
  2.66397837549448e-05,
  1.7042970284819603e-05,
  2.1301209926605225e-05],
 [2.1775253117084503e-05,
  2.5928253307938576e-05,
  3.398861736059189e-05,
  1.7181038856506348e-05,
  2.746284008026123e-05]]

## Testing the second Maxpool layer

In [16]:
# homemade batchNorm1 filter on test
out_homemade = maxpool2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.maxPool2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


[[3.4421682357788086e-06,
  3.635883331298828e-06,
  2.5257468223571777e-06,
  3.606081008911133e-06,
  2.9802322387695312e-06],
 [3.635883331298828e-06,
  3.7550926208496094e-06,
  6.67572021484375e-06,
  1.3709068298339844e-06,
  4.4405460357666016e-06]]

## Testing the Linear layer

In [17]:
# homemade linear filter on test
out_homemade = linear_homemade(torch.reshape(torch.tensor(out_homemade, dtype = torch.double),(2,45)))
# original linear filter on test
out_original = model_original.lin(torch.reshape(out_original, (2,45)))

print("Linear layer 1 error over out channels: ")
compare(out_homemade.numpy(), list(list(out_original.detach().numpy())))

Linear layer 1 error over out channels: 


/home/emil/Documents/bachelor/bachelorprojekt/CNN_layers/linear_layer_homemade.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tmp = torch.dot(input[n],torch.tensor(self.weights[i], dtype = torch.double)) + torch.tensor(self.bias[i], dtype = torch.double)
/home/emil/Documents/bachelor/bachelorprojekt/CNN_layers/linear_layer_homemade.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tmp = torch.hstack((tmp,(torch.dot(input[n],torch.tensor(self.weights[i], dtype = torch.double)) + torch.tensor(self.bias[i], dtype = torch.double))))


[[2.7151122239388314e-07, 1.800861912926166e-06],
 [8.529441899440826e-07, 6.770231237851476e-07]]